In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input , Dense , BatchNormalization, Flatten , LeakyReLU , Reshape
from tensorflow.keras import Model ,Sequential
import matplotlib.pyplot as plt

In [15]:
class GAN():
    def __init__(self):
        self.img_shape= (28,28,1)
        self.latent_dim= 100

        # discriminator
        optimizer = tf.keras.optimizers.Adam(0.0002,0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy',
                                   metrics = ['accuracy'],
                                   optimizer = optimizer)
        
        # Generator
        self.discriminator.trainable = False
        self.generator = self.build_generator()

        z = Input(shape = (self.latent_dim,) )
        g_img = self.generator(z)
        validity = self.discriminator(g_img)

        self.combined = Model(z,validity)
        self.combined.compile(loss='binary_crossentropy',optimizer = optimizer)

    def build_generator(self):

        model = Sequential() 
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2)) 
        model.add(BatchNormalization(momentum=0.8)) 
        model.add(Dense(512)) 
        model.add(LeakyReLU(alpha=0.2)) 
        model.add(BatchNormalization(momentum=0.8)) 
        
        model.add(Dense(1024)) 
        model.add(LeakyReLU(alpha=0.2)) 
        model.add(BatchNormalization(momentum=0.8)) 
        model.add(Dense(np.prod(self.img_shape), activation='tanh')) 
        model.add(Reshape(self.img_shape)) 

        model.summary()
        
        noise = Input(shape=(self.latent_dim,)) 
        img = model(noise) 

        return Model(noise, img)        

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)   

    def train(self,epochs,batch_size=64,sample_interval= 50):

        (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

        # rescale -1 to 1
        x_train=x_train / 127.5 -1.
        x_train= np.reshape(x_train,(-1,28,28,1))

        # Adversarial ground truth
        valid = np.ones((batch_size,1))
        fake = np.zeros((batch_size,1))

        for epoch in range(epochs):

            ######################
            # Train discriminator#
            ######################

            # Discriminator real loss

            idx = np.random.randint(0,x_train.shape[0],batch_size)
            imgs = x_train[idx]
            d_loss_real = self.discriminator.train_on_batch(imgs,valid)

            # Discriminator fake loss

            noise = np.random.normal(0,1,(batch_size,self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs,fake)

            # Average between two losses


            d_loss = np.add(d_loss_real,d_loss_fake) * 0.5

            ##################
            # Train generator#
            ##################

            noise = np.random.normal(0,1,(batch_size,self.latent_dim))

            g_loss = self.combined.train_on_batch(noise,valid)
            
            print('Epoch = %d , D loss = %f , accuracy = %.2f%% / G loss= %f ' % (epoch,d_loss[0],d_loss[1]*100,g_loss))

            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self,epoch):
        r,c = 5,5

        noise = np.random.normal(0,1,(r*c,self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        gen_imgs = 0.5 * gen_imgs +0.5

        fig,axs = plt.subplots(r,c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("./images%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs= 2001, batch_size= 32, sample_interval= 100)






Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_13 (Flatten)        (None, 784)               0         
                                                                 
 dense_91 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_65 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_92 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_66 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_93 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-t

KeyboardInterrupt: ignored

In [14]:
gan = GAN()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_13 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_15 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

TypeError: ignored

In [2]:
def build_generator(img_shape):

    model = Sequential() 
    model.add(Dense(256, input_dim= 100))
    model.add(LeakyReLU(alpha=0.2)) 
    model.add(BatchNormalization(momentum=0.8)) 
    model.add(Dense(512)) 
    model.add(LeakyReLU(alpha=0.2)) 
    model.add(BatchNormalization(momentum=0.8)) 
    
    model.add(Dense(1024)) 
    model.add(LeakyReLU(alpha=0.2)) 
    model.add(BatchNormalization(momentum=0.8)) 
    model.add(Dense(np.prod(img_shape), activation='tanh')) 
    model.add(Reshape(img_shape)) 

    model.summary()
    
    noise = Input(shape=(100,)) 
    img = model(noise) 

    return Model(noise, img)        

def build_discriminator(img_shape):

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)     

In [ ]:
generator = build_generator((28,28,1))
z = Input(shape = (100,) )
g_img = generator(z)
discriminator=build_discriminator((28,28,1))
validity = discriminator(g_img)

In [4]:
discriminator

In [7]:
validity

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'model_1')>